In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel
mode_path = '/root/autodl-tmp/qwen/Qwen2-7B-Instruct/'
lora_path = '/root/loraoutput/Qwen2_instruct_lora2/checkpoint-2000' # 这里改称你的 lora 输出对应 checkpoint 地址

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(mode_path, trust_remote_code=True)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(mode_path, device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True).eval()

# 加载lora权重
model = PeftModel.from_pretrained(model, model_id=lora_path)


In [ ]:
def get_response(prompt: str) -> str:
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

# messages = [
#     {"role": "system", "content": "You are a helpful assistant."},
#     {"role": "user", "content": prompt}
# ]
# text = tokenizer.apply_chat_template(
#     messages,
#     tokenize=False,
#     add_generation_prompt=True
# )
# model_inputs = tokenizer([text], return_tensors="pt").to(device)

# generated_ids = model.generate(
#     model_inputs.input_ids,
#     max_new_tokens=512
# )
# generated_ids = [
#     output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
# ]

# response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]


# 天下没有不散的筵席，那么我们相聚的意义又是什么？
# 咖啡严格来说是不是也可以叫豆浆？
# 石油也是油，为啥没人用它来炒菜？
# 执行死刑时可以委托律师替我去吗？
# 目前我国同性群体离婚率为零，这是否说明同性间结婚比异性更有优势？
# 大家都说我嘴很甜，他们是怎么尝出来的？
# 吃了降压药之后可以摸高压电吗？
# 我爸再婚，我是不是就有了个新娘？
# 为什么我爸妈结婚的时候没邀请我参加婚礼？
# 蓝牙耳机坏了，去医院挂号挂牙科还是耳科？
# 赤道那么暖和，为什么不给东北修一条？
# 为什么长寿的都是老年人？
# 跳多高才能跳过广告？
# 我女朋友的老公应该叫我什么？
# 午餐肉可以在晚餐吃吗？
# 我想配个六千元左右的电脑，大概要多少钱？
# 香菇掉粪坑里了还能叫香菇吗？
# 变形金刚买保险是买车险还是人险？
# 生蚝煮熟了之后叫什么？
# 老鼠生病了，可以吃老鼠药治好吗？
# 去自首的路上被抓了还算自首吗？
# 监狱里罪犯那么多，警察为什么不直接到监狱抓人？
# 地球上的望远镜能看到那么多星球，为什么看不到地球？


In [ ]:
prompt = "天下没有不散的筵席，那么我们相聚的意义又是什么？"
resp = get_response(prompt)